In [ ]:
import requests
import fitz
import pandas as pd
import numpy as np
from multiprocessing.pool import ThreadPool
from urllib.parse import quote

In [ ]:
def get_pdfinfo(entry):
    global timeout, maxlength
    
    #index and url from parquet
    num, url = entry
    error,doc_size,doc_numberofpages,doc_form, characters, links,images,words,blocks = 0,0,0,0,0,0,0,0,0
    
    try:
        with requests.get(url, timeout=timeout) as r:
            if r.status_code == 200:
                doc_size = r.headers["Content-Length"]
                if ((r.headers["Content-Type"] in ["application/pdf","stream/pdf"]) & (int(r.headers["Content-Length"]) < maxlength)):
                    try:
                        doc = fitz.open(stream=r.content, filetype="pdf")
                        #gets the number of pages in the pdf
                        doc_numberofpages = len(doc)
                        
                        #0 if no form element, otherwise the number of form elements in the pdf
                        doc_form = doc.is_form_pdf
                        doc_form = 0 if not doc_form else doc_form
                        
                        #count together characters of readable text, links, images
                        for page in doc:
                            characters += len(page.get_text())
                            links += len(page.get_links())
                            images += len(page.get_images())
                            blocks += len(page.get_textpage().extractBLOCKS())
                            words += len(page.get_textpage().extractWORDS())

                    except:
                        error = "OpenError"
                else:
                    error = "HeaderError"
            else:
                error =  "StatusCodeError"
    except:
        error = "Requesterror"
        
    return [num, error, quote(url, safe='/:?&') ,doc_numberofpages, doc_form, doc_size, characters, links,images,blocks,words]

In [ ]:
# path to parquet file
path = ""
#df = pd.read_parquet(path, engine='pyarrow')
df = pd.read_parquet("http://3080.rom1504.fr/text/text38M/part-00020-a218b5b3-185c-4eb1-956f-fd16cfabd04b-c000.snappy.parquet", engine='pyarrow')

urls = df["url"].tolist()
l = []
for i,url in enumerate(urls):
    a = i, url
    l.append(a)  
    
## global settings for our request
timeout = 3
#max length in bytes of pdf we want to keep
maxlength = 12000000

#get the first X
results = ThreadPool(64).imap_unordered(get_pdfinfo, l[:25])
for info in results:
    if info[1] == 0:
        #[index of parquet, error, quote(url, safe='/:?&') ,doc_numberofpages, doc_form, doc_size, characters, links,images,wordblocks,words]
        print(info)